In [33]:
# OPTIONAL: Load the "autoreload" eX_orig[alias]tension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
import biomart
import os
import re
import matplotlib

import pandas as pd
import pickle as pkl
import numpy as np
import collections as cx
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from goatools.go_search import GoSearch
from scipy import stats

from src.config import get_hussmann_supplementary_xlsx, HOME_PATH, FILTER_COUNT, METHOD, PROFILE_TYPE, get_interim_dir, get_experiment_artifacts

from sklearn.metrics import PrecisionRecallDisplay, jaccard_score

In [35]:
full_results_df = pd.read_pickle(get_experiment_artifacts() + "/augmented_with_GO_outlier_results.pkl")

full_results_df

Sample           Global                                                       \
Measure       Max Score Mean Consistency Mean Score Mean Std CLR Consistency   
Gene                                                                           
0610009B22Rik -0.201933        -0.321429  -0.487445                -0.667511   
0610010K14Rik -0.507595         0.273810  -0.554664                 0.011198   
0610030E20Rik -0.096162         0.130952  -0.334397                 0.255249   
0610040J01Rik  0.587284         0.107143   0.247408                -0.291507   
1110004F10Rik  0.245219        -0.238095   0.024750                -0.283752   
...                 ...              ...        ...                      ...   
Znhit1         0.521303         0.196429   0.292409                 0.798881   
Zranb2         0.955894         0.142857   0.671967                 0.539616   
Zrsr1          0.251615        -0.250000  -0.017171                -1.232652   
Zup1           0.072633        -0.125000  -0.146416                -0.599129   
Zwilch         0.378516         0.071429   0.058289                -0.722994   

Sample                                          T1                         \
Measure       Mean Std Consistency CLR Consistency Consistency  distances   
Gene                                                                        
0610009B22Rik            -1.032894       -0.892857   -0.821429   0.169394   
0610010K14Rik             0.415548        0.392857    0.571429   1.420544   
0610030E20Rik             0.080564        0.178571    0.464286   8.092794   
0610040J01Rik             0.013535        0.000000    0.250000  12.609331   
1110004F10Rik            -0.804892        0.107143   -0.071429   9.343101   
...                            ...             ...         ...        ...   
Znhit1                    0.240788             NaN         NaN        NaN   
Zranb2                    0.106099             NaN         NaN        NaN   
Zrsr1                    -0.803431             NaN         NaN        NaN   
Zup1                     -0.515106             NaN         NaN        NaN   
Zwilch                   -0.056073             NaN         NaN        NaN   

Sample                               T2  ...                        T3  \
Measure         pvalues CLR Consistency  ...   pvalues CLR Consistency   
Gene                                     ...                             
0610009B22Rik  0.999409       -0.071429  ...  0.733467        0.428571   
0610010K14Rik  0.922053        0.071429  ...  0.731721       -0.214286   
0610030E20Rik  0.151195        0.321429  ...  0.684153        0.071429   
0610040J01Rik  0.027328       -0.392857  ...  0.233254        0.285714   
1110004F10Rik  0.096137       -0.071429  ...  0.283318       -0.142857   
...                 ...             ...  ...       ...             ...   
Znhit1              NaN        0.714286  ...  0.019401        0.107143   
Zranb2              NaN        0.142857  ...  0.033617        0.464286   
Zrsr1               NaN       -0.428571  ...  0.400979       -0.464286   
Zup1                NaN       -0.178571  ...  0.116112       -0.178571   
Zwilch              NaN        0.000000  ...  0.372210       -0.464286   

Sample                                         Gene Sets              \
Measure       Consistency  distances   pvalues   Adamson GeneSubset2   
Gene                                                                   
0610009B22Rik   -0.214286   6.214338  0.285918     False       False   
0610010K14Rik    0.428571   2.872796  0.719589     False       False   
0610030E20Rik   -0.214286   3.770737  0.582873     False       False   
0610040J01Rik    0.142857  14.842168  0.011058     False       False   
1110004F10Rik   -0.428571  11.102672  0.049382     False       False   
...                   ...        ...       ...       ...         ...   
Znhit1          -0.500000   9.116254  0.104516      True       False   
Zranb2           0.464286  18.871878  0.002

In [36]:
pseudo_controls = pd.read_csv(get_experiment_artifacts() + "/psuedo_controls.txt").rename(columns={"0": "Gene"})["Gene"]
pseudo_controls.shape

(100,)

In [37]:
column_mappings = {
    "pvalues": "P-value",
    "distances": "Score",
    "Mean Score": "Combined score"
}

final_outlier_df = full_results_df.rename(columns=column_mappings, level=1)
final_outlier_df = final_outlier_df.rename(columns={"Global": "Overall"}, level=0)

idx = pd.IndexSlice
final_outlier_df = final_outlier_df.loc[:, final_outlier_df.columns.get_level_values("Measure").isin(list(column_mappings.values()))]

final_outlier_df[("Overall", "Rank")] = final_outlier_df[("Overall", "Combined score")].rank(ascending=False)

final_outlier_df = final_outlier_df.sort_values(("Overall", "Rank"))

final_outlier_df[("Overall", "Pseudo-control")] = final_outlier_df.index.to_series().isin(pseudo_controls)

correct_order = []
for a in ["T1", "T2", "T3"]:
    for b in ["Score"]:
        correct_order.append((a, b))
correct_order = correct_order + [('Overall',   'Combined score'),
            ('Overall',             'Rank'),
            ('Overall',   'Pseudo-control')]

final_outlier_df = final_outlier_df[correct_order]            

final_outlier_df.to_csv("artifacts/final_gene_list.tsv", sep="\t")

final_outlier_df.columns.names = [None, None]

final_outlier_df


T1          T2           T3        Overall           \
               Score       Score        Score Combined score     Rank   
Gene                                                                    
Atp6v1g1         NaN  191.691909  1639.034599      83.465198      1.0   
H2ac18           NaN   79.209169   417.400779      22.147886      2.0   
Metap2           NaN  143.007300   338.365098      21.620708      3.0   
Xrcc5            NaN  130.088007   319.605493      20.137448      4.0   
H2ax      253.225165  259.034827   177.472082      19.940038      5.0   
...              ...         ...          ...            ...      ...   
Tmcc2       1.424668    0.600808     0.991567      -0.682672  18074.0   
Speg        1.099064    0.453060     1.067054      -0.694064  18075.0   
Espn        0.726430    0.349433     0.973139      -0.710487  18076.0   
Cdc14a      0.894752    0.519442     0.523577      -0.714095  18077.0   
Gnas        0.967983    0.401086     0.204367      -0.725639  18078.0   

                         
         Pseudo-control  
Gene                     
Atp6v1g1          False  
H2ac18            False  
Metap2            False  
Xrcc5             False  
H2ax              False  
...                 ...  
Tmcc2              True  
Speg               True  
Espn               True  
Cdc14a             True  
Gnas               True  

[18078 rows x 6 columns]

In [38]:
get_experiment_artifacts(profile_type = PROFILE_TYPE)

'/Users/colm/repos/MUSICian/notebooks/exploratory/outlier_analysis/artifacts/paired-replicates.robust_cov.700/'

In [39]:
X = pkl.load(open("{}/imputed_sample_data.pkl".format(get_experiment_artifacts(profile_type = PROFILE_TYPE)), "rb"))

print("Data Ready.")

Data Ready.


In [40]:
for a in X:
    X[a].to_csv("artifacts/final_paired_replicate_mutational_profile_{}.tsv".format(a), sep="\t")

In [41]:
for c in sorted(X[a].columns):
    print(c)


Any Insertion
Deletion 0bp microhomology
Deletion 1bp microhomology
Deletion 2bp microhomology
Deletion 3+bp microhomology
Deletion with insertion
Homology Directed Repair


In [42]:
method = pd.DataFrame({
    "Method": [METHOD],
    "Profile Type": [PROFILE_TYPE],
    "Minimum Read Count": [FILTER_COUNT]
})
method


,Method,Profile Type,Minimum Read Count
0,robust_cov,paired-replicates,700


In [43]:
writer = pd.ExcelWriter('artifacts/final_gene_list.xlsx', engine='xlsxwriter')

final_outlier_df.to_excel(writer, sheet_name="final_gene_list")

for a in X:
    X[a][sorted(X[a].columns)].to_excel(writer, sheet_name="final_mutational_profile_{}".format(a))

method.to_excel(writer, sheet_name="config")

writer.close()